In [36]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from sklearn.model_selection import train_test_split

In [37]:
def load_data():
    # loading and preprocessing dataset 
    data = pd.read_csv("../dataset/annotations/annotations.csv")
    data = data[['id','image','score']]
        
    images = []
    for i in range(0,len(data)):
        filepath = data['image'][i]
        image = cv2.imread('../dataset/images/{0}'.format(filepath),cv2.IMREAD_COLOR)

        if image is not None:
            new_dimensions = (224, 224)
            resized_image = cv2.resize(image, new_dimensions, interpolation=cv2.INTER_AREA)
            images.append(resized_image)

    return np.array(images), np.array(data['score'])

In [38]:
images, ratings = load_data()
images = images / 255.0  # normalize images

print(images.shape)

X_train, X_test, y_train, y_test = train_test_split(images, ratings, test_size=0.2, random_state=42)

print(y_train)


(107, 224, 224, 3)
[82. 29. 57. 57. 79. 54. 79. 57. 57. 79. 61. 57. 43. 64. 82. 57. 46. 61.
 79. 57. 57. 68. 75. 39. 57. 54. 82. 71. 79. 57. 75. 57. 71. 71. 79. 57.
 79. 57. 79. 54. 75. 61. 50. 75. 57. 79. 79. 57. 57. 36. 82. 79. 57. 57.
 64. 79. 79. 57. 79. 79. 79. 71. 79. 57. 71. 64. 57. 57. 57. 61. 57. 57.
 57. 82. 79. 79. 79. 57. 57. 79. 79. 75. 57. 57. 79.]


In [39]:
base_model = ResNet50(
    weights='imagenet', 
    include_top=False, 
    input_shape=(224,224,3)
    )

# Add custom top layers for regression
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='linear')(x)  # Output layer for regression

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, batch_size=32)


Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - loss: 4183.7070 - mae: 63.3772 - val_loss: 3090.7395 - val_mae: 54.5420
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 674ms/step - loss: 2820.4883 - mae: 51.3857 - val_loss: 2063.1604 - val_mae: 44.1280
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 693ms/step - loss: 1828.1238 - mae: 40.8848 - val_loss: 1241.9486 - val_mae: 33.5472
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 684ms/step - loss: 1098.5206 - mae: 30.7080 - val_loss: 608.9020 - val_mae: 22.1506
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 667ms/step - loss: 560.8367 - mae: 20.6446 - val_loss: 228.8410 - val_mae: 10.9997
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 698ms/step - loss: 263.9670 - mae: 12.8880 - val_loss: 125.0555 - val_mae: 10.3456
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 671ms/step - loss: 226.2657 - mae: 12.5732 - val_loss: 223.4444 - val_mae: 12.8113
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 678ms/step - loss: 304.0238 - mae: 13.6639 - val_loss: 348.7752 - val_mae: 15.5205
Epoch 9/20
3/3 ━━━━━

In [40]:
for layer in base_model.layers[-10:]:
    layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='mean_squared_error', metrics=['mae'])
history_fine = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32)

test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - loss: 195.2019 - mae: 11.2811 - val_loss: 121.8218 - val_mae: 10.3612
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 729ms/step - loss: 178.4998 - mae: 10.8208 - val_loss: 121.4980 - val_mae: 10.3928
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 752ms/step - loss: 223.9911 - mae: 12.4717 - val_loss: 121.3020 - val_mae: 10.4308
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 728ms/step - loss: 166.1949 - mae: 10.7280 - val_loss: 121.3700 - val_mae: 10.4719
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 701ms/step - loss: 162.2874 - mae: 10.1488 - val_loss: 121.5427 - val_mae: 10.5075
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 762ms/step - loss: 158.8095 - mae: 10.1180 - val_loss: 121.9601 - val_mae: 10.5448
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 760ms/step - loss: 158.7991 - mae: 10.4966 - val_loss: 122.3893 - val_mae: 10.5740
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 765ms/step - loss: 160.8922 - mae: 10.4408 - val_loss: 123.0144 - val_mae: 10.6032
Epoch 9/10
3/3 ━━━━━━━━━━━━

In [41]:
model.save("BetterSEOModel.h5")

In [42]:
new_model = tf.keras.models.load_model('BetterSEOModel.h5')

In [43]:
# Predict rating for a new screenshot
def predict_rating(screenshot):
    new_dimensions = (224,224)
    resized_image = cv2.resize(screenshot, new_dimensions, interpolation=cv2.INTER_AREA)
    screenshot = resized_image / 255.0
    screenshot = np.expand_dims(screenshot, axis=0)  
    predicted_rating = new_model.predict(screenshot)
    return predicted_rating[0][0]

# Load Screenshot and make prediction
new_screenshot = cv2.imread('../dataset/images/{0}'.format('architecturalrecord.png'),cv2.IMREAD_COLOR)
predicted_rating = predict_rating(new_screenshot)
print(f'Predicted Rating: {predicted_rating}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 785ms/step
Predicted Rating: 71.59967041015625
